# This file can be used to parse through the marine cadastre website for years 2018-2020

In [7]:
import pandas as pd
import dask.dataframe as dd
import glob
import math
#REMEMBER ONLY CLICK ON THIS ONCE
max_lat = []
min_lat = []
max_lon = []
min_lon = []
max_sog = []
min_sog = []
max_cog = []
min_cog = []


directory = glob.glob(r"E:\Capstone\New_AIS_Data\2020\*.csv")

In [3]:
for file in directory:
    cols = ["MMSI", "BaseDateTime", "LAT", "LON", "SOG", "COG", "VesselType", "Status"]
    train_df = dd.read_csv(file, parse_dates = ['BaseDateTime'], dtype={'Cargo': 'float64','VesselType': 'float64', 'LAT': 'float32', 'LON': 'float32', 'SOG': 'float32', 'COG': 'float32'}, usecols = cols)
    train_df = train_df.compute()
    #drop all rows not in Zone 19
    train_df["Zone"] = train_df["LON"].apply(lambda x: (math.floor((x+180)/6) % 60) +1)
    train_df = train_df[train_df['Zone'] == 19]
    #drop the rows where there's no data for 'MMSI', 'BaseDateTime', 'LAT', 'SOG', 'LON', 'COG', 'VesselType'
    train_df = train_df.dropna(subset=['MMSI', 'BaseDateTime', 'LAT', 'SOG', 'LON', 'COG', 'VesselType'])
    #Change datatypes to shrink size of data frame
    train_df['MMSI'] = pd.to_numeric(train_df['MMSI'], downcast='unsigned')
    train_df['LAT'] = pd.to_numeric(train_df['LAT'], downcast='float')
    train_df['LON'] = pd.to_numeric(train_df['LON'], downcast='float')
    train_df['SOG'] = pd.to_numeric(train_df['SOG'], downcast='float')
    train_df['COG'] = pd.to_numeric(train_df['COG'], downcast='float')
    train_df['VesselType'] = pd.to_numeric(train_df['VesselType'], downcast='unsigned')


    #Arrange in order of the MMSI number
    train_df = train_df.sort_values(by=['MMSI', 'BaseDateTime'])

    #remove any improper MMSIs (those whose length isn't 9)
    train_df.MMSI = train_df.MMSI.astype('str')
    train_df['MMLength'] = train_df.MMSI.str.len()
    index_names = train_df[ train_df['MMLength'] != 9 ].index 
    train_df.drop(index_names, inplace = True)
    train_df = train_df.drop('MMLength', axis = 1)

    #remove those at anchor and moored
    index_names2 = train_df[ train_df['Status'] == 'at anchor' ].index 
    train_df.drop(index_names2, inplace = True)
    index_names3 = train_df[ train_df['Status'] == 'moored' ].index
    train_df.drop(index_names3, inplace = True)
    train_df = train_df.drop('Status', axis = 1)

    #remove those with improper vessel types
    index_names3 = train_df[ train_df['VesselType'] == 0].index
    train_df.drop(index_names3, inplace = True)

    #remove rows that only occur once
    train_df = train_df[train_df.groupby('MMSI').MMSI.transform(len) > 99]


    #reset the indices
    train_df = train_df.reset_index(drop = True)



    #convert Vessel Type to categories
    # 0=Cargo, 1=Fishing, 2=Tug Tow, 3=Other, 4=Passenger, 5=PleasureCraft/Sailing, 6=Tanker, 7=Military
    preserved_titles = [1004,1003,70,71,72,73,74,75,76,77,78,79,30,1001,1002,1021,60,61,62,63,64,65,66,67,68,69,1012,1013,1014,1015,1019,36,37,
                       80,81,82,83,84,85,86,87,88,89,1024,21,22,31,32,52,1023,1025,20,23,24,25,26,27,28,29,40,41,42,43,44,45,46,47,48,49]
    train_df.loc[~train_df['VesselType'].isin(preserved_titles), 'VesselType'] = 3
    train_df['VesselType'].replace([1004,1003,70,71,72,73,74,75,76,77,78,79],0, inplace = True)
    train_df['VesselType'].replace([30,1001,1002],1, inplace = True)
    train_df['VesselType'].replace([1021],7, inplace = True)
    train_df['VesselType'].replace([60,61,62,63,64,65,66,67,68,69,1012,1013,1014,1015],4, inplace = True)
    train_df['VesselType'].replace([1019,36,37],5, inplace = True)
    train_df['VesselType'].replace([80,81,82,83,84,85,86,87,88,89,1024],6, inplace = True)
    train_df['VesselType'].replace([21,22,31,32,52,1023,1025],2, inplace = True)
    train_df['VesselType'].replace([20,23,24,25,26,27,28,29],3, inplace = True)
    train_df['VesselType'].replace([40,41,42,43,44,45,46,47,48,49],3, inplace = True)
    train_df['VesselType'] = pd.to_numeric(train_df['VesselType'], downcast='unsigned')

    #get number of categories
    #if you remove the .index from the next line you get a series, which you can normalize to get a percentage of data
    cat_num = train_df['VesselType'].value_counts().index



    train_df['MMSI'] = pd.to_numeric(train_df['MMSI'], downcast='unsigned')
    
    max_lat_train = train_df['LAT'].max()
    min_lat_train = train_df['LAT'].min()
    max_lon_train = train_df['LON'].max()
    min_lon_train = train_df['LON'].min()
    max_sog_train = train_df['SOG'].max()
    min_sog_train = train_df['SOG'].min()
    max_cog_train = train_df['COG'].max()
    min_cog_train = train_df['COG'].min()
    
    
    max_lat.append(max_lat_train)
    min_lat.append(min_lat_train)
    max_lon.append(max_lon_train)
    min_lon.append(min_lon_train)
    max_sog.append(max_sog_train)
    min_sog.append(min_sog_train)
    max_cog.append(max_cog_train)
    min_cog.append(min_cog_train)
    
    train_df = train_df.drop('Zone', axis = 1)
    train_df.to_csv (r"E:\Capstone\New_AIS_Data\Shrunken_AIS_Data_2020\{}".format(file[30:]), index = False, header=True)

In [21]:
print("max_lat")
print(max(max_lat))
print("max_lon")
print(max(max_lon))
print("max_sog")
print(max(max_sog))
print("max_cog")
print(max(max_cog))
print("min_lat")
print(min(min_lat))
print("min_lon")
print(min(min_lon))
print("min_sog")
print(min(min_sog))
print("min_cog")
print(min(min_cog))

max_lat
89.96446
max_lon
-66.00001
max_sog
51.1
max_cog
204.7
min_lat
0.0001
min_lon
-72.0
min_sog
-51.2
min_cog
-204.8
